In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Import
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

# **Area calculation for each land use category in each station**

In [ ]:
#### area for each landuse id for each station 
df = pd.read_csv("/content/drive/My Drive/Thesis-data/station_landuse.csv",low_memory=False)
df.reset_index(inplace = True, drop = True)

#### data for using final label from GIS and survey?
data4 = pd.read_csv("/content/drive/My Drive/Thesis-data/Subway_station_labling_wlanduse.csv",low_memory=False)

In [ ]:
#@title Calculating the area for each land-use category
df2 = df.copy()
df2['residential'] = df2['1'] + df2['22']
df2['commercial'] = df2['2'] + df2['20']
df2['education'] = df2['3'] + df2['4']
df2['hospital'] = df2['8'] + df2['9']
df2['work'] = df2['10'] + df2['21']
df2['recreational'] = df2['5'] + df2['6'] + df2['7'] + df2['17'] + df2['18'] + df2['11']
df2['other'] = df2['12'] + df2['13'] + df2['14'] + df2['15'] + df2['24'] + df2['19'] + df2['-1']

c3 = ['residential','commercial','education','hospital','work','recreational','other','station']
df2 = df2.iloc[:][c3]

In [ ]:
#@title Adding final label to data
df3 = df2.merge(data4, left_on= 'station', right_on= 'STATION_CODE')
df3 = df3.drop('STATION_CODE',1)

# **Importing the trip data (activity)**

In [ ]:
data_all = pd.read_csv("/content/drive/My Drive/Thesis-data/data_all_with_frequency_10000_1398-11_extra.csv",low_memory=False)
data_all = data_all.drop(['index', 'home_location', 'indx','stations','distance'],1)

data_other = pd.read_csv("/content/drive/My Drive/Thesis-data/CH3_F_10000_other(F)_1398-11_extra.csv",low_memory=False)
data_other = data_other.drop(['index', 'home_location', 'indx','z','stations','distance'],1)

In [ ]:
#@title Merging the lnad-use data with activites data
data_other = data_other.merge(df3, left_on= 'd', right_on= 'station')
data_other = data_other.drop('station',1)
data_all = data_all.merge(df3, left_on= 'd', right_on= 'station')
data_all = data_all.drop('station',1)

**Other category activity sampling**

In [ ]:
data_w = data_all[data_all['z']==0]
data_e = data_w[data_w['Final_label']=='Education']
data_e = data_e[data_e['duration'] < 8]
data_e = data_e[data_e['time d'] > 8]

In [ ]:
df_1 = data_other[data_other['Final_label']=='Commercial'].sample(n=1000)
df_2 = data_other[data_other['Final_label']=='Recreational'].sample(n=1000)
df_3 = data_other[data_other['Final_label']=='Hospital'].sample(n=100)
df_4 = data_other[data_other['Final_label']=='Education'].sample(n=800)
df_4_2 = data_e.sample(n=100)
df_5 = data_other[data_other['Final_label']=='Other'].sample(n=1000)

**Home/Work category activity sampling**

In [ ]:
# df_6 = data_all[data_all['z']==0].sample(n=1000)
# df_7 = data_all[data_all['z']==1].sample(n=1000)

**Merging the data**

In [ ]:
#### OTHER ####
df_f = df_1.append(df_2)
df_f = df_f.append(df_3)
df_f = df_f.append(df_4)
df_f = df_f.append(df_4_2)
df_f = df_f.append(df_5)

df_f = df_f.drop('z',1)
#### HOME/WORK ####
# df_f = df_f.append(df_6)
# df_f = df_f.append(df_7)

#### Imbalanced ####
df_f = data_other
#df_f = df_f.append(df_4_2)
# df_f = df_4_2

df_f.reset_index(inplace = True, drop = True)
print(len(df_f))

54343


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


**Normalizing the row (area variable)**

In [ ]:
data_m = df_f.copy()
data_m = data_m.drop(['work','residential'],1)

c = ['commercial','education','hospital','recreational']
data_m['total'] = data_m.iloc[:][c].sum(axis=1)

data_n = data_m.copy()
data_n['commercial'] = data_m['commercial']/data_m['total']
data_n['education'] = data_m['education']/data_m['total']
data_n['hospital'] = data_m['hospital']/data_m['total']
data_n['recreational'] = data_m['recreational']/data_m['total']


data_n['total'] = data_n.iloc[:][c].sum(axis=1)
data_n.head()

# **Saving the dataset**

In [ ]:
data_n.to_csv("/content/drive/My Drive/Thesis-data/data-LDA-other_without_other_imbalanced_1398_11_extra.csv",index=False)